In [2]:
import pandas as pd
import zipfile
import os
import struct
from pathlib import Path

In [6]:
csv = pd.read_csv("./BTCUSDT/2025-07/BTCUSDT-trades-2025-07-01.csv")
csv.head()

,5055476813,107146.51000000,0.00160000,171.43441600,1751328000015562,False,True
0,5055476814,107146.5,0.01117,1196.826405,1751328000178512,True,True
1,5055476815,107146.5,0.00005,5.357325,1751328000178512,True,True
2,5055476816,107146.5,0.00005,5.357325,1751328000178512,True,True
3,5055476817,107146.5,0.15328,16423.415520,1751328000178512,True,True
4,5055476818,107146.5,0.00049,52.501785,1751328000178512,True,True


In [3]:
def convertToBoolean(value: str):
    return 1 if str(value).lower() in ['true', '1'] else 0

In [ ]:
directory = Path("BTCUSDT/raw")


# Price: Operations Prices
# QTY: Amount negotiated
# Time: Exact time operation
# IsBuyerMarker: If the buyer put the ticket first
use = {1: 'price', 2: 'qty', 4: 'time', 5: 'isBuyerMaker'}

for zip_path in sorted(directory.glob("*.zip")):
    print(zip_path.name)

    with zipfile.ZipFile(zip_path, 'r') as Z:
        print(f"This is namelist: ", Z.namelist())
        csv_names = [n for n in Z.namelist() if n.lower().endswith(".csv")]
        print("This is csv: ", csv_names)
        for csv_name in csv_names:
            with Z.open(csv_name) as f:
                df = pd.read_csv(f, 
                                 header=None, 
                                 usecols=list(use),
                                 dtype={1: 'float64', 2: 'float64', 4: 'Int64'},
                                 converters={5: convertToBoolean}
                                 )
                df = df.rename(columns=use)[['time', 'price', 'qty', 'isBuyerMaker']]
                if not df["time"].is_monotonic_increasing:
                    raise ValueError("The colummn is not in increasing order")

                for row in df.itertuples():
                    print("Time: ", row.time, " Price: ", row.price, " QTY: ", row.qty, " isBuyerMarker: ", row.isBuyerMaker)
                    break

        break       

BTCUSDT-trades-2025-07-01.zip
This is namelist:  ['BTCUSDT-trades-2025-07-01.csv']
This is csv:  ['BTCUSDT-trades-2025-07-01.csv']
Time:  1751328000015562  Price:  107146.51  QTY:  0.0016  isBuyerMarker:  0
